In [0]:
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.tokenize import sent_tokenize, word_tokenize
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import movie_reviews
from nltk import pos_tag
from nltk.corpus import wordnet
from google.colab import files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
lemmatizer = WordNetLemmatizer()

In [31]:
files.upload()

Saving TSA_training_twitter_x_y_train.csv to TSA_training_twitter_x_y_train (1).csv


{'TSA_training_twitter_x_y_train.csv': b'tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone\n567900433542488064,negative,Southwest,,ColeyGirouard,,0,"@SouthwestAir I am scheduled for the morning, 2 days after the fact, yes..not sure why my evening flight was the only one Cancelled Flightled",,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)\n569989168903819264,positive,Southwest,,WalterFaddoul,,0,@SouthwestAir seeing your workers time in and time out going above and beyond is why I love flying with you guys. Thank you!,,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)\n568089179520954368,positive,United,,LocalKyle,,0,"@united Flew ORD to Miami and back and  had great crew, service on both legs. THANKS",,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)\n568928195581513728,negative,Southwest,,amccarthy19,,0,@SouthwestAir @

In [7]:
files.upload()

Saving TSA_test_twitter_x_test.csv to TSA_test_twitter_x_test.csv


{'TSA_test_twitter_x_test.csv': b'tweet_id,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone\n569682010270101504,American,,zsalim03,,0,@AmericanAir In car gng to DFW. Pulled over 1hr ago - very icy roads. On-hold with AA since 1hr. Can\'t reach arpt for AA2450. Wat 2 do?,,2015-02-22 18:15:50 -0800,Texas,Central Time (US & Canada)\n569608307184242688,American,,sa_craig,,0,"@AmericanAir after all, the plane didn\xe2\x80\x99t land in identical or worse) conditions at GRK according to METARs.",,2015-02-22 13:22:57 -0800,"College Station, TX",Central Time (US & Canada)\n567879304593408001,Southwest,,DanaChristos,,1,@SouthwestAir can\'t believe how many paying customers you left high and dry with no reason for flight Cancelled Flightlations Monday out of BDL! Wow.,,2015-02-17 18:52:31 -0800,CT,Eastern Time (US & Canada)\n569757651539660801,US Airways,,rossj987,,0,@USAirways I can legitimately say that I would hav

In [8]:
nltk.download('stopwords')
Stopword = stopwords.words('english')
punctuation = list(string.punctuation)
Stopword = Stopword + punctuation
print(Stopword)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'bo

In [0]:
train_pd = pd.read_csv("TSA_training_twitter_x_y_train.csv")
test_pd = pd.read_csv("TSA_test_twitter_x_test.csv")

In [10]:
train_pd.head()

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [11]:
test_pd.head()

,tweet_id,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,569682010270101504,American,NaN,zsalim03,NaN,0,@AmericanAir In car gng to DFW. Pulled over 1h...,NaN,2015-02-22 18:15:50 -0800,Texas,Central Time (US & Canada)
1,569608307184242688,American,NaN,sa_craig,NaN,0,"@AmericanAir after all, the plane didn’t land ...",NaN,2015-02-22 13:22:57 -0800,"College Station, TX",Central Time (US & Canada)
2,567879304593408001,Southwest,NaN,DanaChristos,NaN,1,@SouthwestAir can't believe how many paying cu...,NaN,2015-02-17 18:52:31 -0800,CT,Eastern Time (US & Canada)
3,569757651539660801,US Airways,NaN,rossj987,NaN,0,@USAirways I can legitimately say that I would...,NaN,2015-02-22 23:16:24 -0800,"Washington, D.C.",Eastern Time (US & Canada)
4,569900705852608513,American,NaN,tranpham18,NaN,0,@AmericanAir still no response from AA. great ...,NaN,2015-02-23 08:44:51 -0800,New York City,Eastern Time (US & Canada)


In [15]:
nltk.download('punkt')
tweets = []
for index, rows in train_pd.iterrows():
    tweets.append((rows['text'],rows['airline_sentiment']))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
tweet_test = []
for index, rows in test_pd.iterrows():
    tweet_test.append((rows['text']))

In [16]:
print(tweets[0:5])

[('@SouthwestAir I am scheduled for the morning, 2 days after the fact, yes..not sure why my evening flight was the only one Cancelled Flightled', 'negative'), ('@SouthwestAir seeing your workers time in and time out going above and beyond is why I love flying with you guys. Thank you!', 'positive'), ('@united Flew ORD to Miami and back and  had great crew, service on both legs. THANKS', 'positive'), ("@SouthwestAir @dultch97 that's horse radish 😤🐴", 'negative'), ('@united so our flight into ORD was delayed because of Air Force One, but the last flight to SBN is at 8:20, 5 mins from now we just landed.', 'negative')]


In [0]:
tweets = [(word_tokenize(text), sentiment) for text, sentiment in tweets]

In [0]:
tweet_test = [(word_tokenize(text)) for text in tweet_test]

In [18]:
print(tweets[0:5])

[(['@', 'SouthwestAir', 'I', 'am', 'scheduled', 'for', 'the', 'morning', ',', '2', 'days', 'after', 'the', 'fact', ',', 'yes..not', 'sure', 'why', 'my', 'evening', 'flight', 'was', 'the', 'only', 'one', 'Cancelled', 'Flightled'], 'negative'), (['@', 'SouthwestAir', 'seeing', 'your', 'workers', 'time', 'in', 'and', 'time', 'out', 'going', 'above', 'and', 'beyond', 'is', 'why', 'I', 'love', 'flying', 'with', 'you', 'guys', '.', 'Thank', 'you', '!'], 'positive'), (['@', 'united', 'Flew', 'ORD', 'to', 'Miami', 'and', 'back', 'and', 'had', 'great', 'crew', ',', 'service', 'on', 'both', 'legs', '.', 'THANKS'], 'positive'), (['@', 'SouthwestAir', '@', 'dultch97', 'that', "'s", 'horse', 'radish', '😤🐴'], 'negative'), (['@', 'united', 'so', 'our', 'flight', 'into', 'ORD', 'was', 'delayed', 'because', 'of', 'Air', 'Force', 'One', ',', 'but', 'the', 'last', 'flight', 'to', 'SBN', 'is', 'at', '8:20', ',', '5', 'mins', 'from', 'now', 'we', 'just', 'landed', '.'], 'negative')]


In [0]:
def remove_stopwords(word_list):
    new_word_list = []
    for w in word_list:
        if w.lower() not in Stopword:
            new_word_list.append(w)
    return new_word_list

In [0]:
tweets = [(remove_stopwords(text), sentiment) for text, sentiment in tweets]

In [0]:
tweet_test = [(remove_stopwords(text)) for text in tweet_test]

In [21]:
print(tweets[0:5])

[(['SouthwestAir', 'scheduled', 'morning', '2', 'days', 'fact', 'yes..not', 'sure', 'evening', 'flight', 'one', 'Cancelled', 'Flightled'], 'negative'), (['SouthwestAir', 'seeing', 'workers', 'time', 'time', 'going', 'beyond', 'love', 'flying', 'guys', 'Thank'], 'positive'), (['united', 'Flew', 'ORD', 'Miami', 'back', 'great', 'crew', 'service', 'legs', 'THANKS'], 'positive'), (['SouthwestAir', 'dultch97', "'s", 'horse', 'radish', '😤🐴'], 'negative'), (['united', 'flight', 'ORD', 'delayed', 'Air', 'Force', 'One', 'last', 'flight', 'SBN', '8:20', '5', 'mins', 'landed'], 'negative')]


In [0]:
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('R'):
        return wordnet.ADV
    else: return wordnet.NOUN

In [0]:
def clean_tweets(words):
    output_word = []
    for w in words:
        if w not in Stopword:
            pos = pos_tag([w])
            new_word = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1]))
            output_word.append(new_word.lower())
    return output_word

In [26]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
tweets = [(clean_tweets(text), sentiment) for text, sentiment in tweets]

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
tweet_test = [(clean_tweets(text)) for text in tweet_test]

In [27]:
print(tweets[0:5])

[(['southwestair', 'schedule', 'morning', '2', 'day', 'fact', 'yes..not', 'sure', 'even', 'flight', 'one', 'cancelled', 'flightled'], 'negative'), (['southwestair', 'see', 'worker', 'time', 'time', 'go', 'beyond', 'love', 'fly', 'guy', 'thank'], 'positive'), (['united', 'flew', 'ord', 'miami', 'back', 'great', 'crew', 'service', 'leg', 'thanks'], 'positive'), (['southwestair', 'dultch97', "'s", 'horse', 'radish', '😤🐴'], 'negative'), (['united', 'flight', 'ord', 'delayed', 'air', 'force', 'one', 'last', 'flight', 'sbn', '8:20', '5', 'min', 'land'], 'negative')]


In [28]:
all_words = []
for text, sentiment in tweets:
    all_words += text
words = set(all_words)
print(len(words))

13105


In [0]:
sentiments = [sentiment for text, sentiment in tweets]

In [0]:
tweet = [" ".join(text) for text, sentiments in tweets]

In [0]:
tweet_tests = [" ".join(text) for text in tweet_test]

In [35]:
count_vec = CountVectorizer(max_features = 300)
tweet_features = count_vec.fit_transform(tweet)
tweet_features.todense()

matrix([[0, 0, 0, ..., 1, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [46]:
print(count_vec.get_feature_names())

['10', '15', '1st', '20', '30', '45', 'aa', 'able', 'add', 'agent', 'ago', 'air', 'airline', 'airport', 'almost', 'already', 'also', 'always', 'americanair', 'amp', 'another', 'answer', 'anyone', 'anything', 'app', 'appreciate', 'arrive', 'ask', 'attendant', 'available', 'awesome', 'back', 'bad', 'bag', 'baggage', 'best', 'board', 'boarding', 'book', 'booking', 'bos', 'business', 'ca', 'call', 'cancelled', 'card', 'care', 'change', 'charge', 'check', 'checked', 'chicago', 'claim', 'class', 'clt', 'co', 'come', 'connect', 'connection', 'contact', 'could', 'credit', 'crew', 'customer', 'dallas', 'day', 'dca', 'deal', 'delay', 'delayed', 'delta', 'departure', 'dfw', 'direct', 'disappointed', 'dm', 'do', 'due', 'earlier', 'early', 'email', 'employee', 'even', 'ever', 'every', 'ewr', 'experience', 'extra', 'fail', 'family', 'fee', 'finally', 'find', 'first', 'fix', 'fleek', 'fleet', 'flight', 'flighted', 'flightled', 'flightr', 'flt', 'fly', 'follow', 'free', 'friend', 'frustrate', 'full', 

In [0]:
tweet_test_features = count_vec.transform(tweet_tests)

In [47]:
svc = SVC()
svc.fit(tweet_features,sentiments)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [0]:
solution = svc.predict(tweet_test_features)

In [54]:
print(solution)

['negative' 'negative' 'negative' ... 'neutral' 'positive' 'negative']


In [0]:
with open('TSA_svm.csv', 'wb') as f:
    np.savetxt(f, solution, fmt='%s', delimiter=",")

files.download('TSA_svm.csv')

In [58]:
rf = RandomForestClassifier()
rf.fit(tweet_features,sentiments)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [0]:
solution_rf = rf.predict(tweet_test_features)

In [0]:
with open('TSA_rf.csv', 'wb') as f:
    np.savetxt(f, solution, fmt='%s', delimiter=",")

files.download('TSA_rf.csv')